# 載入模組

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import pandas as pd

# 讀取檔案
## 利用未除息來訓練,除息後來檢定(認為市場交易的實際價格為為除息的價格,較能反應市場)

In [2]:
test = pd.read_csv("taetfp.csv", error_bad_lines=False)
train = pd.read_csv("tetfp.csv", error_bad_lines=False)
#submit = pd.read_csv('gender_submission.csv')

# 改變欄位名稱

In [3]:
test.columns=['code','date','name','open','high','low','close','volume']
train.columns=['code','date','name','open','high','low','close','volume']


# 查看資料型態

In [4]:
train.head()

,code,date,name,open,high,low,close,volume
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,16487
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,29020
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,9837
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,8910
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,12507


In [5]:
test.head()

,code,date,name,open,high,low,close,volume
0,50,20130102,元大台灣50,46.57,47.13,46.49,46.92,16487
1,50,20130103,元大台灣50,47.35,47.48,47.13,47.31,29020
2,50,20130104,元大台灣50,47.31,47.31,46.92,47.00,9837
3,50,20130107,元大台灣50,47.05,47.05,46.49,46.79,8910
4,50,20130108,元大台灣50,46.57,46.75,46.27,46.49,12507


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19054 entries, 0 to 19053
Data columns (total 8 columns):
code      19054 non-null int64
date      19054 non-null int64
name      19054 non-null object
open      19054 non-null float64
high      19054 non-null float64
low       19054 non-null float64
close     19054 non-null float64
volume    19054 non-null int64
dtypes: float64(4), int64(3), object(1)
memory usage: 1.2+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19053 entries, 0 to 19052
Data columns (total 8 columns):
code      19053 non-null int64
date      19053 non-null int64
name      19053 non-null object
open      19053 non-null float64
high      19053 non-null float64
low       19053 non-null float64
close     19053 non-null float64
volume    19053 non-null int64
dtypes: float64(4), int64(3), object(1)
memory usage: 1.2+ MB


In [8]:
train.describe()

,code,date,open,high,low,close,volume
count,19054.000000,1.905400e+04,19054.000000,19054.000000,19054.000000,19054.000000,19054.000000
mean,1763.966201,2.015329e+07,32.832552,32.931248,32.720399,32.823267,898.025349
std,2727.620365,1.571827e+04,14.079738,14.117945,14.039040,14.080184,3697.839312
min,50.000000,2.013010e+07,9.300000,9.710000,9.240000,9.250000,0.000000
25%,53.000000,2.014053e+07,23.360000,23.450000,23.270000,23.360000,4.000000
50%,57.000000,2.015101e+07,31.580000,31.690000,31.485000,31.555000,21.000000
75%,6201.000000,2.017031e+07,39.900000,39.990000,39.777500,39.850000,180.000000
max,6208.000000,2.018043e+07,88.050000,88.400000,87.900000,88.300000,290124.000000


In [9]:
test.describe()

,code,date,open,high,low,close,volume
count,19053.000000,1.905300e+04,19053.000000,19053.000000,19053.000000,19053.000000,19053.000000
mean,1764.055844,2.015329e+07,30.928367,31.020185,30.823855,30.919575,897.983835
std,2727.663879,1.571746e+04,13.591398,13.624949,13.555332,13.591569,3697.931917
min,50.000000,2.013010e+07,8.690000,9.070000,8.630000,8.640000,0.000000
25%,53.000000,2.014053e+07,20.950000,21.020000,20.880000,20.960000,4.000000
50%,57.000000,2.015101e+07,29.560000,29.630000,29.450000,29.540000,21.000000
75%,6201.000000,2.017031e+07,37.970000,38.050000,37.880000,37.960000,180.000000
max,6208.000000,2.018043e+07,85.840000,86.180000,85.690000,86.080000,290124.000000


# 計算技術指標

In [10]:
import talib
import numpy
def talib2df(talib_output):
    if type(talib_output) == list:
        ret = pd.DataFrame(talib_output).transpose()
    else:
        ret = pd.Series(talib_output)
    ret.index = train['close'].index
    return ret;

In [11]:
t1 = {
    'close':train.close.dropna().astype(float),
    'open':train.open.dropna().astype(float),
    'high':train.high.dropna().astype(float),
    'low':train.low.dropna().astype(float),
    'volume': train.volume.dropna().astype(float)    
}

# 採用 KD/MACD/RSI指標

In [12]:
KD = talib2df(talib.abstract.STOCH(t1, fastk_period=9))

In [13]:
MACD = talib2df(talib.abstract.MACD(t1))

In [14]:
RSI = talib2df(talib.abstract.RSI(t1))

In [15]:
t1=pd.DataFrame(t1)
t1 = pd.concat([train,KD,MACD,RSI], axis=1)

In [16]:
t1.columns=['code','date','name','open','high','low','close','volume','k','d','dif12','dif26','macd','rsi']

In [19]:
t1.head(50)

,code,date,name,open,high,low,close,volume,k,d,dif12,dif26,macd,rsi
33,50,20130226,元大台灣50,55.25,55.40,55.05,55.10,6753,45.418886,58.302230,0.385145,0.343851,0.041294,51.348701
34,50,20130227,元大台灣50,55.05,55.20,54.95,55.20,4605,28.628763,45.589086,0.349906,0.345062,0.004844,52.613598
35,50,20130301,元大台灣50,55.35,55.55,55.30,55.40,5730,20.115942,31.387864,0.334264,0.342903,-0.008639,55.126444
36,50,20130304,元大台灣50,55.40,55.45,54.70,54.75,8270,19.777778,22.840827,0.266348,0.327592,-0.061244,46.496626
37,50,20130305,元大台灣50,54.90,55.35,54.90,55.20,16081,25.456790,21.783503,0.245999,0.311273,-0.065274,52.088591
38,50,20130306,元大台灣50,55.70,55.75,55.20,55.45,4640,34.290123,26.508230,0.247196,0.298458,-0.051262,54.908225
39,50,20130307,元大台灣50,55.20,55.65,55.20,55.40,5956,52.623457,37.456790,0.241328,0.287032,-0.045704,54.220942
40,50,20130308,元大台灣50,55.60,55.90,55.60,55.80,12380,70.833333,52.582305,0.265889,0.282803,-0.016914,58.677128
41,50,20130311,元大台灣50,55.80,56.10,55.60,55.90,12066,78.571429,67.342740,0.290080,0.284259,0.005821,59.732430
42,50,20130312,元大台灣50,55.90,56.00,55.55,55.55,7442,79.365079,76.256614,0.277806,0.282968,-0.005162,54.487544


# 20130225前的技術指標值存在空值,所以移除

In [18]:
t1 = t1.query('date > 20130225')

In [25]:
import numpy as np

from sklearn import model_selection, ensemble, preprocessing, metrics

### 移除遺漏值、與設漲即為1

In [26]:
df=t1
df['label']=(df.close - df.close.shift(1)) > 0
df=df.dropna()

### 創造 dummy variables

In [27]:
label_encoder = preprocessing.LabelEncoder()
encoded_label = label_encoder.fit_transform(df["label"])

# 區分不同標的

In [28]:
tra50 = df.query('code == 50')
tra51 = df.query('code == 51')
tra52 = df.query('code == 52')
tra53 = df.query('code == 53')
tra54 = df.query('code == 54')
tra55 = df.query('code == 55')
tra56 = df.query('code == 56')
tra57 = df.query('code == 57')
tra58 = df.query('code == 58')
tra59 = df.query('code == 59')
tra690 = df.query('code == 690')
tra692 = df.query('code == 692')
tra701 = df.query('code == 701')
tra713 = df.query('code == 713')
tra6201 = df.query('code == 6201')
tra6203 = df.query('code == 6203')
tra6204 = df.query('code == 6204')
tra6208 = df.query('code == 6208')

### 測試還有建立的數據

In [29]:
selected_features = ['code','date','open','high','low','close','volume','k','d','macd','rsi']
t1_X = df[selected_features]
t1_y = df['label']
train_X, test_X, train_y, test_y = model_selection.train_test_split(t1_X, t1_y, test_size = 0.3)

### 建立 random forest 模型

In [30]:
forest = ensemble.RandomForestClassifier(n_estimators = 150)
forest_fit = forest.fit(train_X, train_y)

### 預測

In [31]:
test_y_predicted = forest.predict(test_X)
#test_y_predicted

### 績效

In [32]:
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
#print(accuracy)
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
#print(auc)
print('準確率: {}'.format(auc))
print('AUC值: {}'.format(accuracy))

準確率: 0.7018103688275265
AUC值: 0.7025815704553603


In [33]:
tra50['diff']=(tra50.close - tra50.close.shift(1))

C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# 預測股價走向

In [34]:
import numpy as np
today_X = tra50[selected_features]
today_y_predicted = forest.predict(today_X)
proba = forest.predict_proba(today_X)


In [35]:
print('明日走勢: ' + format(np.where(today_y_predicted==True,'漲','跌')[0]))

明日走勢: 跌


# 利用近月來的漲跌幅度來預測每期的漲跌幅度

In [36]:
fluct = tra50.query('date > 20180301')
posfl = fluct.query('diff > 0')
negfl = fluct.query('diff < 0')

In [37]:
rise = posfl['diff'].mean()
drop = negfl['diff'].mean()

In [38]:
tra50['close'].tail(1)

1302    79.2
Name: close, dtype: float64

In [40]:
if format(np.where(today_y_predicted == True,1,-1)[0]) == 1:
    print (tra50['close'].tail(1) + rise)
else:
    print (tra50['close'].tail(1) + drop)

1302    78.652273
Name: close, dtype: float64


# 利用除權息資料預測

In [41]:
import sys
import talib
import numpy
def talib2df2(talib_output):
    if type(talib_output) == list:
        ret = pd.DataFrame(talib_output).transpose()
    else:
        ret = pd.Series(talib_output)
    ret.index = test['close'].index
    return ret;

In [42]:
t2 = {
    'close':test.close.dropna().astype(float),
    'open':test.open.dropna().astype(float),
    'high':test.high.dropna().astype(float),
    'low':test.low.dropna().astype(float),
    'volume': test.volume.dropna().astype(float)    
}

In [43]:
KD = talib2df2(talib.abstract.STOCH(t2, fastk_period=9))
MACD = talib2df2(talib.abstract.MACD(t2))
RSI = talib2df2(talib.abstract.RSI(t2))

In [44]:
t2=pd.DataFrame(t2)
t2= pd.concat([train,KD,MACD,RSI], axis=1)
t2.columns=['code','date','name','open','high','low','close','volume','k','d','dif12','dif26','macd','rsi']
t2 = t2.dropna()
#顯示結果
final_X = t2[selected_features]
final_y_predicted = forest.predict(final_X)
proba = forest.predict_proba(final_X)
print('明日走勢: {}'.format(np.where(final_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

明日走勢: 跌
明日漲的機率: 0.18666666666666668


In [45]:
aim =[50,51,52,53,54,55,57,58,59,690,692,701,713,6201,6203,6204,6208]

In [46]:
fin50 = t2.query('code == 50')
fin51 = t2.query('code == 51')
fin52 = t2.query('code == 52')
fin53 = t2.query('code == 53')
fin54 = t2.query('code == 54')
fin55 = t2.query('code == 55')
fin56 = t2.query('code == 56')
fin57 = t2.query('code == 57')
fin58 = t2.query('code == 58')
fin59 = t2.query('code == 59')
fin690 = t2.query('code == 690')
fin692 = t2.query('code == 692')
fin701 = t2.query('code == 701')
fin713 = t2.query('code == 713')
fin6201 = t2.query('code == 6201')
fin6203 = t2.query('code == 6203')
fin6204 = t2.query('code == 6204')
fin6208 = t2.query('code == 6208')


In [47]:
fin50_X = fin50[selected_features]
fin50_y_predicted = forest.predict(fin50_X)
proba = forest.predict_proba(fin50_X)
print('明日走勢: {}'.format(np.where(fin50_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin50['diff']=(fin50.close - fin50.close.shift(1))

fluct50 = fin50.query('date > 20180301')
posfl50 = fluct50.query('diff > 0')
negfl50 = fluct50.query('diff < 0')

rise = posfl50['diff'].mean()
drop = negfl50['diff'].mean()

if format(np.where(fin50_y_predicted == True,1,-1)[0]) == 1:
    print (fin50['close'].tail(1) + rise)
else:
    print (fin50['close'].tail(1) + drop)

明日走勢: 跌
明日漲的機率: 0.18666666666666668
1302    78.652273
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [50]:
fin51_X = fin51[selected_features]
fin51_y_predicted = forest.predict(fin51_X)
proba = forest.predict_proba(fin51_X)
print('明日走勢: {}'.format(np.where(fin51_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin51['diff']=(fin51.close - fin51.close.shift(1))

fluct51 = fin51.query('date > 20180301')
posfl51 = fluct51.query('diff > 0')
negfl51 = fluct51.query('diff < 0')

rise = posfl51['diff'].mean()
drop = negfl51['diff'].mean()

if format(np.where(fin51_y_predicted == True,1,-1)[0]) == 1:
    print (fin51['close'].tail(1) + rise)
else:
    print (fin51['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.20666666666666667
2605    31.895
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [51]:
fin52_X = fin52[selected_features]
fin52_y_predicted = forest.predict(fin52_X)
proba = forest.predict_proba(fin52_X)
print('明日走勢: {}'.format(np.where(fin52_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin52['diff']=(fin52.close - fin52.close.shift(1))

fluct52 = fin52.query('date > 20180301')
posfl52 = fluct52.query('diff > 0')
negfl52 = fluct52.query('diff < 0')

rise = posfl52['diff'].mean()
drop = negfl52['diff'].mean()

if format(np.where(fin52_y_predicted == True,1,-1)[0]) == 1:
    print (fin52['close'].tail(1) + rise)
else:
    print (fin52['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.24666666666666667
3908    52.605263
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [52]:
fin53_X = fin53[selected_features]
fin53_y_predicted = forest.predict(fin53_X)
proba = forest.predict_proba(fin53_X)
print('明日走勢: {}'.format(np.where(fin53_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin53['diff']=(fin53.close - fin53.close.shift(1))

fluct53 = fin53.query('date > 20180301')
posfl53 = fluct53.query('diff > 0')
negfl53 = fluct53.query('diff < 0')

rise = posfl53['diff'].mean()	
drop = negfl53['diff'].mean()

if format(np.where(fin53_y_predicted == True,1,-1)[0]) == 1:
    print (fin53['close'].tail(1) + rise)
else:
    print (fin53['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.14
5211    33.88
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [53]:
fin54_X = fin54[selected_features]
fin54_y_predicted = forest.predict(fin54_X)
proba = forest.predict_proba(fin54_X)
print('明日走勢: {}'.format(np.where(fin54_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin54['diff']=(fin54.close - fin54.close.shift(1))

fluct54 = fin54.query('date > 20180301')
posfl54 = fluct54.query('diff > 0')
negfl54 = fluct54.query('diff < 0')

rise = posfl54['diff'].mean()	
drop = negfl54['diff'].mean()

if format(np.where(fin54_y_predicted == True,1,-1)[0]) == 1:
    print (fin54['close'].tail(1) + rise)
else:
    print (fin54['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.19333333333333333
6514    22.956667
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [54]:
fin55_X = fin55[selected_features]
fin55_y_predicted = forest.predict(fin55_X)
proba = forest.predict_proba(fin55_X)
print('明日走勢: {}'.format(np.where(fin55_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin55['diff']=(fin55.close - fin55.close.shift(1))

fluct55 = fin55.query('date > 20180301')
posfl55 = fluct55.query('diff > 0')
negfl55 = fluct55.query('diff < 0')

rise = posfl55['diff'].mean()	
drop = negfl55['diff'].mean()

if format(np.where(fin55_y_predicted == True,1,-1)[0]) == 1:
    print (fin55['close'].tail(1) + rise)	
else:
    print (fin55['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.43333333333333335
7817    16.964444
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [55]:
fin56_X = fin56[selected_features]
fin56_y_predicted = forest.predict(fin56_X)
proba = forest.predict_proba(fin56_X)
print('明日走勢: {}'.format(np.where(fin56_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin56['diff']=(fin56.close - fin56.close.shift(1))

fluct56 = fin56.query('date > 20180301')
posfl56 = fluct56.query('diff > 0')
negfl56 = fluct56.query('diff < 0')

rise = posfl56['diff'].mean()	
drop = negfl56['diff'].mean()

if format(np.where(fin56_y_predicted == True,1,-1)[0]) == 1:
    print (fin56['close'].tail(1) + rise)	
else:
    print (fin56['close'].tail(1) + drop)


明日走勢: 漲
明日漲的機率: 0.7933333333333333
9121    25.047826
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [56]:
fin57_X = fin57[selected_features]
fin57_y_predicted = forest.predict(fin57_X)
proba = forest.predict_proba(fin57_X)
print('明日走勢: {}'.format(np.where(fin57_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin57['diff']=(fin57.close - fin57.close.shift(1))

fluct57 = fin57.query('date > 20180301')
posfl57 = fluct57.query('diff > 0')
negfl57 = fluct57.query('diff < 0')

rise = posfl57['diff'].mean()	
drop = negfl57['diff'].mean()

if format(np.where(fin57_y_predicted == True,1,-1)[0]) == 1:
    print (fin57['close'].tail(1) + rise)	
else:
    print (fin57['close'].tail(1) + drop)


明日走勢: 漲
明日漲的機率: 0.82
10424    48.32
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [57]:
fin58_X = fin58[selected_features]
fin58_y_predicted = forest.predict(fin58_X)
proba = forest.predict_proba(fin58_X)
print('明日走勢: {}'.format(np.where(fin58_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin58['diff']=(fin58.close - fin58.close.shift(1))

fluct58 = fin58.query('date > 20180301')
posfl58 = fluct58.query('diff > 0')
negfl58 = fluct58.query('diff < 0')

rise = posfl58['diff'].mean()	
drop = negfl58['diff'].mean()

if format(np.where(fin58_y_predicted == True,1,-1)[0]) == 1:
    print (fin58['close'].tail(1) + rise)	
else:
    print (fin58['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.19333333333333333
11727    44.597143
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [58]:
fin59_X = fin59[selected_features]
fin59_y_predicted = forest.predict(fin59_X)
proba = forest.predict_proba(fin59_X)
print('明日走勢: {}'.format(np.where(fin59_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin59['diff']=(fin59.close - fin59.close.shift(1))

fluct59 = fin59.query('date > 20180301')
posfl59 = fluct59.query('diff > 0')
negfl59 = fluct59.query('diff < 0')

rise = posfl59['diff'].mean()	
drop = negfl59['diff'].mean()

if format(np.where(fin59_y_predicted == True,1,-1)[0]) == 1:
    print (fin59['close'].tail(1) + rise)	
else:
    print (fin59['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.20666666666666667
13030    41.725
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [59]:
fin690_X = fin690[selected_features]
fin690_y_predicted = forest.predict(fin690_X)
proba = forest.predict_proba(fin690_X)
print('明日走勢: {}'.format(np.where(fin690_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin690['diff']=(fin690.close - fin690.close.shift(1))

fluct690 = fin690.query('date > 20180301')
posfl690 = fluct690.query('diff > 0')
negfl690 = fluct690.query('diff < 0')

rise = posfl690['diff'].mean()	
drop = negfl690['diff'].mean()

if format(np.where(fin690_y_predicted == True,1,-1)[0]) == 1:
    print (fin690['close'].tail(1) + rise)	
else:
    print (fin690['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.14
18507    21.424286
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [60]:
fin692_X = fin692[selected_features]
fin692_y_predicted = forest.predict(fin692_X)
proba = forest.predict_proba(fin692_X)
print('明日走勢: {}'.format(np.where(fin692_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin692['diff']=(fin692.close - fin692.close.shift(1))

fluct692 = fin692.query('date > 20180301')
posfl692 = fluct692.query('diff > 0')
negfl692 = fluct692.query('diff < 0')

rise = posfl692['diff'].mean()	
drop = negfl692['diff'].mean()

if format(np.where(fin692_y_predicted == True,1,-1)[0]) == 1:
    print (fin692['close'].tail(1) + rise)	
else:
    print (fin692['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.2
18742    20.701579
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [61]:
fin701_X = fin701[selected_features]
fin701_y_predicted = forest.predict(fin701_X)
proba = forest.predict_proba(fin701_X)
print('明日走勢: {}'.format(np.where(fin701_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin701['diff']=(fin701.close - fin701.close.shift(1))

fluct701 = fin701.query('date > 20180301')
posfl701 = fluct701.query('diff > 0')
negfl701 = fluct701.query('diff < 0')

rise = posfl701['diff'].mean()	
drop = negfl701['diff'].mean()

if format(np.where(fin701_y_predicted == True,1,-1)[0]) == 1:
    print (fin701['close'].tail(1) + rise)	
else:
    print (fin701['close'].tail(1) + drop)


明日走勢: 漲
明日漲的機率: 0.68
18912    20.757619
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [62]:
fin713_X = fin713[selected_features]
fin713_y_predicted = forest.predict(fin713_X)
proba = forest.predict_proba(fin713_X)
print('明日走勢: {}'.format(np.where(fin713_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin713['diff']=(fin713.close - fin713.close.shift(1))

fluct713 = fin713.query('date > 20180301')
posfl713 = fluct713.query('diff > 0')
negfl713 = fluct713.query('diff < 0')

rise = posfl713['diff'].mean()	
drop = negfl713['diff'].mean()	

if format(np.where(fin713_y_predicted == True,1,-1)[0]) == 1:
    print (fin713['close'].tail(1) + rise)	
else:
    print (fin713['close'].tail(1) + drop)


明日走勢: 漲
明日漲的機率: 0.8333333333333334
19052    29.954118
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [63]:
fin6201_X = fin6201[selected_features]
fin6201_y_predicted = forest.predict(fin6201_X)
proba = forest.predict_proba(fin6201_X)
print('明日走勢: {}'.format(np.where(fin6201_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin6201['diff']=(fin6201.close - fin6201.close.shift(1))

fluct6201 = fin6201.query('date > 20180301')
posfl6201 = fluct6201.query('diff > 0')
negfl6201 = fluct6201.query('diff < 0')

rise = posfl6201['diff'].mean()	
drop = negfl6201['diff'].mean()	

if format(np.where(fin6201_y_predicted == True,1,-1)[0]) == 1:
    print (fin6201['close'].tail(1) + rise)	
else:
    print (fin6201['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.38
14333    13.05375
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [64]:
fin6203_X = fin6203[selected_features]
fin6203_y_predicted = forest.predict(fin6203_X)
proba = forest.predict_proba(fin6203_X)
print('明日走勢: {}'.format(np.where(fin6203_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin6203['diff']=(fin6203.close - fin6203.close.shift(1))

fluct6203 = fin6203.query('date > 20180301')
posfl6203 = fluct6203.query('diff > 0')
negfl6203 = fluct6203.query('diff < 0')

rise = posfl6203['diff'].mean()	
drop = negfl6203['diff'].mean()	

if format(np.where(fin6203_y_predicted == True,1,-1)[0]) == 1:
    print (fin6203['close'].tail(1) + rise)	
else:
    print (fin6203['close'].tail(1) + drop)


明日走勢: 漲
明日漲的機率: 0.7666666666666667
15636    36.455333
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [65]:
fin6204_X = fin6204[selected_features]
fin6204_y_predicted = forest.predict(fin6204_X)
proba = forest.predict_proba(fin6204_X)
print('明日走勢: {}'.format(np.where(fin6204_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin6204['diff']=(fin6204.close - fin6204.close.shift(1))

fluct6204 = fin6204.query('date > 20180301')
posfl6204 = fluct6204.query('diff > 0')
negfl6204 = fluct6204.query('diff < 0')

rise = posfl6204['diff'].mean()	
drop = negfl6204['diff'].mean()	

if format(np.where(fin6204_y_predicted == True,1,-1)[0]) == 1:
    print (fin6204['close'].tail(1) + rise)	
else:
    print (fin6204['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.24
16939    51.715385
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [66]:
fin6208_X = fin6208[selected_features]
fin6208_y_predicted = forest.predict(fin6208_X)
proba = forest.predict_proba(fin6208_X)
print('明日走勢: {}'.format(np.where(fin6208_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

fin6208['diff']=(fin6208.close - fin6208.close.shift(1))

fluct6208 = fin6208.query('date > 20180301')
posfl6208 = fluct6208.query('diff > 0')
negfl6208 = fluct6208.query('diff < 0')

rise = posfl6208['diff'].mean()	
drop = negfl6208['diff'].mean()	

if format(np.where(fin6208_y_predicted == True,1,-1)[0]) == 1:
    print (fin6208['close'].tail(1) + rise)	
else:
    print (fin6208['close'].tail(1) + drop)


明日走勢: 跌
明日漲的機率: 0.16666666666666666
18242    45.731905
Name: close, dtype: float64


C:\Users\x555l\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
